In [1]:
import pickle
import pandas as pd
import numpy as np
from pgmpy.models import LinearGaussianBayesianNetwork
from pgmpy.factors.continuous import LinearGaussianCPD
from pgmpy.inference.ExactInference import BeliefPropagation
from scipy.stats import multivariate_normal
from pgmpy.factors.continuous import ContinuousFactor
from pgmpy.models import BayesianModel
import numpy.linalg as la
import emcee
from tqdm import tqdm_notebook

In [2]:
## DAG

PC_DAG = pd.read_csv('data/DAG_PC.csv',index_col = 'Unnamed: 0')
with open('data/MLE_PC.pkl','rb') as f:
    PC_MLE = pickle.load(f)
    
with open('data/means.pkl','rb') as f:
    mean = pickle.load(f)
    
mean_ser = pd.Series(np.array(mean).mean(0), index = PC_DAG.columns)

g = LinearGaussianBayesianNetwork()

g.add_nodes_from(PC_DAG.columns)

edges = [(PC_DAG.index[edge[0]], PC_DAG.columns[edge[1]]) for edge in np.argwhere(PC_DAG.values)]

g.add_edges_from(edges)

for factor in PC_MLE:
    name = factor[0]
    beta = factor[1]
    var = factor[2]
    pars = factor[3]
    if len(pars)==0:
        cpd = LinearGaussianCPD(name, [], var, pars)
        cpd.beta_0 = mean_ser[name]
        g.add_cpds(cpd)
    else:
        cpd = LinearGaussianCPD(name,  beta, var, pars)
        cpd.beta_0 = mean_ser[name]
        g.add_cpds(cpd)

norm = g.to_joint_gaussian()

In [3]:
# MRF
mrf = pd.read_csv('data/MRF.csv',index_col = 'Unnamed: 0')
sigma_mrf = la.inv(mrf.values)
mu_mrf = np.array(mean).mean(0)
vars_mrf = list(PC_DAG.columns)

In [4]:
vars_dag = norm.variables
mu_dag = pd.Series(mu_mrf,index=vars_mrf).reindex(vars_dag)
sigma_dag = norm.covariance


In [5]:
def lnprob(x, mu, cov,icov, conditions, vars = None):
    for condition in conditions:
        ind = vars.index(condition)
        x[ind] = conditions[condition]
    k = len(x)
    diff = x-mu
    term1 = np.log(la.det(cov))
    term2 = np.dot(diff,np.dot(icov,diff))
    term3 = k*np.log(2*np.pi)
    return -1/2*(term1 + term2 + term3)

In [6]:
def mcmc(mu,sigma,conditions,vars):
    ndim = 51
    nwalkers = 250
    p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[mu,sigma, la.inv(sigma), conditions, vars])
    pos, prob, state = sampler.run_mcmc(p0, 100)
    sampler.reset()
    sampler.run_mcmc(pos, 1000)
    print(np.mean(sampler.acceptance_fraction))
    return pd.Series(sampler.flatchain.mean(0),index = vars)

def mh(mu,sigma,conditions,vars):
    ndim = 51
    nwalkers = 250
    p0 = np.random.rand(ndim)
    sampler = emcee.MHSampler(sigma, ndim, lnprob, args=[mu,sigma, la.inv(sigma), conditions, vars])
    pos, prob, state = sampler.run_mcmc(p0, 1000)
    sampler.reset()
    sampler.run_mcmc(pos, 10000)
    print(np.mean(sampler.acceptance_fraction))
    return pd.Series(sampler.flatchain.mean(0),index = vars)
    

In [7]:
election = pd.read_csv('data/results.csv',index_col='State')
temp = list(election.index)

temp[-1] = 'District of Columbia'
election.index = temp

In [8]:
def pred_clinton_mcmc(mu,sigma,vars,time = 6.5):
    evidence = election[election.Poll_Closing <= time].Clinton.to_dict()
    vote_perc = mcmc(mu_dag, sigma_dag, evidence,vars_dag)
    clinton_win = (vote_perc>.50).astype(int).reindex(election.index)
    return vote_perc.loc[['Florida','Michigan','Pennsylvania','Wisconsin']], (clinton_win*election['number of votes']).sum()

In [9]:
def pred_clinton_mh(mu,sigma,vars,time = 6.5):
    evidence = election[election.Poll_Closing <= time].Clinton.to_dict()
    vote_perc = mh(mu_dag, sigma_dag, evidence,vars_dag)
    clinton_win = (vote_perc>.50).astype(int).reindex(election.index)
    return vote_perc.loc[['Florida','Michigan','Pennsylvania','Wisconsin']], (clinton_win*election['number of votes']).sum()

In [11]:
times = np.unique(election.Poll_Closing)
times = [6.5] + list(times)
for model in tqdm_notebook(['dag','mrf']):
    for time in tqdm_notebook(times):
        swings = []
        ecs = []
        for i in tqdm_notebook(range(10)):
            if model == 'dag':
                swing, ec = pred_clinton_mcmc(mu_dag,sigma_dag,vars_dag,time)
            elif model == 'mrf':
                swing, ec = pred_clinton_mcmc(mu_mrf,sigma_mrf,vars_mrf,time)
            title = 'data/{}_GW_{}.pkl'.format(model,time)
            swings.append(swing)
            ecs.append(ec)
        ec = np.mean(ecs)
        swing = pd.Series(np.array(swings).mean(0),index = swing.index)
        with open(title, 'wb') as f:
            pickle.dump((swing,ec), f)

0.203232
0.20196799999999998
0.202888
0.20445600000000003
0.203408
0.207488
0.20246
0.20386400000000002
0.20257600000000003
0.20518


0.20330400000000004
0.20429200000000003
0.20129600000000003
0.20382800000000004
0.207512
0.200696
0.20368
0.20380800000000002
0.202844
0.20650400000000002


0.202256
0.203024
0.20262400000000003
0.202052
0.20024799999999998
0.203012
0.20579599999999998
0.20213599999999998
0.201828
0.20521999999999999


0.20439600000000002
0.20416399999999998
0.202972
0.203768
0.20542399999999997
0.203904
0.204492
0.20402
0.204912
0.20384400000000003


0.20338
0.20435200000000003
0.204068
0.20686000000000002
0.20324
0.20468
0.20392000000000002
0.20303200000000002
0.203724
0.204504


0.2347
0.23417600000000002
0.231352
0.232792
0.23210000000000003
0.23319600000000001
0.23200800000000002
0.23286400000000004
0.231712
0.23200800000000002


0.21256799999999998
0.241072
0.23745600000000003
0.242532
0.22704400000000002
0.211308
0.229572
0.21857200000000002
0.21606000000000003
0.23932


0.601012
0.59848
0.600692
0.5994839999999999
0.597668
0.598136
0.599296
0.599932
0.5991439999999999
0.60042


In [12]:
for model in tqdm_notebook(['dag','mrf']):
    for time in tqdm_notebook(times):
        swings = []
        ecs = []
        for i in tqdm_notebook(range(10)):
            if model == 'dag':
                swing, ec = pred_clinton_mh(mu_dag,sigma_dag,vars_dag,time)
            elif model == 'mrf':
                swing, ec = pred_clinton_mh(mu_mrf,sigma_mrf,vars_mrf,time)
            title = 'data/{}_MH_{}.pkl'.format(model,time)

            swings.append(swing)
            ecs.append(ec)
        ec = np.mean(ecs)
        swing = pd.Series(np.array(swings).mean(0),index = swing.index)
        with open(title, 'wb') as f:
            pickle.dump((swing,ec), f)

0.0017
0.0041
0.0029
0.0032
0.0041
0.0025
0.0022
0.0024
0.0029
0.0042


0.0029
0.0017
0.0042
0.0021
0.0042
0.0023
0.0021
0.0034
0.0046
0.0046


0.0059
0.0023
0.0038
0.0042
0.0057
0.0014
0.0054
0.0055
0.0029
0.0041


0.0055
0.011
0.0146
0.0076
0.0127
0.0096
0.0112
0.0067
0.0055
0.0102


0.0134
0.0123
0.0107
0.0097
0.0106
0.0077
0.0132
0.0113
0.012
0.0098


...

0.0135
0.0143
0.0105
0.0106
0.0124
0.0102
0.0061
0.0087
0.0099
0.0107


0.0125
0.0116
0.009
0.0124
0.0125
0.0105
0.0132
0.0093
0.014
0.0064


0.0981
0.0923
0.0918
0.1049
0.1045
0.0939
0.0939
0.0947
0.0992
0.097


0.6661
0.6707
0.6746
0.6641
0.6739
0.675
0.6696
0.6705
0.6717
0.6684


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
